In [1]:
!pip install transformers
!pip install sentencepiece

  Using cached transformers-4.12.3-py3-none-any.whl (3.1 MB)
  Using cached filelock-3.3.2-py3-none-any.whl (9.7 kB)
  Using cached huggingface_hub-0.1.2-py3-none-any.whl (59 kB)
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached regex-2021.11.2-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (749 kB)
  Using cached tokenizers-0.10.3-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached sentencepiece-0.1.96-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)


In [2]:
import pandas as pd
import numpy as np

import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import config
from transformers import XLMRobertaForTokenClassification, XLMRobertaConfig ,BertModel, XLMRobertaTokenizer, XLMRobertaModel, BertForTokenClassification
from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
from seqeval.metrics import accuracy_score, classification_report
import engine
from model_new import EntityModel
import torch.nn as nn
from tqdm import tqdm

  Using cached seqeval-1.2.2-py3-none-any.whl
Object `nn.Linear` not found.


In [3]:
# Read the data
def process_data(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags):
    
    dataObj = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
    df= dataObj.createDf() # get dataframe from files
    obj_tokenized = createTokenizedDf(df)
    df_new= obj_tokenized.convertDf()
    enc_label = preprocessing.LabelEncoder()
    df_new['labels']= enc_label.fit_transform(df_new['labels'])
    train_data = CompDataset(df_new)
    return train_data


In [4]:
dataset = process_data(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags)
len(dataset)

7000

In [5]:
dataset[0]

(tensor([     0,  17151,  12426,   2765,    113,    527, 110896,  36997,     71,
          20387,   2189,    141,     99, 183124,     23,  58020,      6,      5,
              2,      2,    656,  58020,    656, 138438,     13,    656,  17151,
            656,  12426,   2765,    656,    113,    656,    527, 110896,    656,
          20387,   2189,    141,    656,     23,    656, 183124,    656,      6,
              5,    656,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,    

In [6]:
train_data, test_data= model_selection.train_test_split(dataset, random_state = 34, test_size=0.1)

In [7]:
len(train_data)

6300

In [8]:
class createDataloaders:
    
    def __init__(self,dataset,batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        
    def createDataloaders(self):
        data_loader = torch.utils.data.DataLoader(self.dataset,batch_size = self.batch_size ,shuffle = True, num_workers = 2)  # for data to be returned in batches for batch grad-descent
    # one fold is now divided into 4 batches that can be accessed with any iterator like for ,etc
        return data_loader

In [10]:
# dataloaders
loader_obj = createDataloaders(train_data,10)
train_dataloader = loader_obj.createDataloaders()
train_dataloader

In [11]:
len(train_dataloader)

630

In [12]:
loader_obj = createDataloaders(test_data,10)
val_dataloader = loader_obj.createDataloaders()

In [13]:
len(val_dataloader)

70

In [15]:
batch = next(iter(train_dataloader))
batch[0]

[2021-11-09 14:13:24.930 pytorch-1-6-gpu-p-ml-g4dn-12xlarge-5c3490cc3803aef3e3308aaea43f:40 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-09 14:13:24.962 pytorch-1-6-gpu-p-ml-g4dn-12xlarge-5c3490cc3803aef3e3308aaea43f:40 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


tensor([[    0,  1672,   361,  ...,     1,     1,     1],
        [    0,  2363,  4568,  ...,     1,     1,     1],
        [    0,  4911,     7,  ...,     1,     1,     1],
        ...,
        [    0,  5337, 61340,  ...,     1,     1,     1],
        [    0,   764,   509,  ...,     1,     1,     1],
        [    0,    98, 18237,  ...,     1,     1,     1]])

RuntimeError: both arguments to matmul need to be at least 1D, but they are 0D and 2D

In [16]:
def loss_fn(output,target,mask):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
    active_logits = output.view(-1,2)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)    
    )
    loss = lfn(active_logits,active_labels)
    return loss

In [17]:
class EntityModel(nn.Module):
    
    def __init__(self):
        
        super(EntityModel, self).__init__()
        self.bert = XLMRobertaForTokenClassification.from_pretrained(config.BASE_MODEL,output_attentions = False, output_hidden_states = False)
#         self.bert_drop_1 = nn.Dropout(0.3)
#         self.out_tag = nn.Linear(768, 2)
    
    def forward(self, ids, mask, labels):
        
        outputs = self.bert(ids,
                                attention_mask = mask,
                                labels = labels,return_dict=False)
#         bo_tag = self.bert_drop_1(output_1)
        
#         tag = self.out_tag(bo_tag)  
        
        loss_tag = loss_fn(outputs[1],labels,mask)
        
#         return bo_tag,loss
        return outputs
        

In [18]:
model = EntityModel()
model.cuda()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

EntityModel(
  (bert): XLMRobertaForTokenClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_

In [40]:
b_input_ids = batch[0].cuda()
b_input_mask = batch[1].cuda()
b_labels = batch[2].cuda()
outputs = model(b_input_ids, 
                b_input_mask,
                labels=b_labels)

In [44]:
# outputs

In [18]:
class EntityModel(nn.Module):
    
    def __init__(self):
        
        super(EntityModel, self).__init__()
        self.bert = XLMRobertaModel.from_pretrained(config.BASE_MODEL,output_attentions = False, output_hidden_states = False)
        self.bert_drop_1 = nn.Dropout(0.1)
        self.out_tag = nn.Linear(768, 2)
    
    def forward(self, ids, mask, labels):
        
        outputs = self.bert(ids,
                                attention_mask = mask,
                                return_dict=False)
        bo_tag = self.bert_drop_1(outputs[0])
        
        tag = self.out_tag(bo_tag)  
        
        loss_tag = loss_fn(tag,labels,mask)
        
        return tag,loss_tag
#         return tag
        

In [18]:
model = EntityModel()
model.cuda()
b_input_ids = batch[0].cuda()
b_input_mask = batch[1].cuda()
b_labels = batch[2].cuda()
outputs = model(b_input_ids, 
                b_input_mask,
                labels=b_labels)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
# torch.cuda.memory_summary(device=None, abbreviated=False)
# torch.cuda.empty_cache()
outputs[0]

tensor([[[-0.2872,  0.0848],
         [ 0.1034,  0.3015],
         [-0.0435, -0.0179],
         ...,
         [-0.3093,  0.0745],
         [-0.2578,  0.1114],
         [-0.1465,  0.0035]],

        [[-0.0442, -0.0580],
         [-0.2054,  0.5346],
         [-0.2299,  0.6233],
         ...,
         [-0.2199,  0.1799],
         [-0.1488, -0.2768],
         [-0.0408, -0.1451]],

        [[ 0.0228, -0.0703],
         [-0.3343,  0.4891],
         [ 0.0216,  0.1470],
         ...,
         [-0.0400, -0.1608],
         [-0.3054,  0.1259],
         [-0.2510,  0.1900]],

        [[-0.2303,  0.1715],
         [-0.3593,  0.0989],
         [ 0.0885,  0.2508],
         ...,
         [-0.3375,  0.0201],
         [-0.2023,  0.1925],
         [-0.2374,  0.1316]]], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
transformers.XLMRobertaForTokenClassification.from_pretrained(config.BASE_MODEL)

In [19]:
param_optimizer = list(model.named_parameters())

In [20]:
model =EntityModel()
model.cuda()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EntityModel(
  (bert): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [20]:
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
num_train_steps = int(len(train_data) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)

In [21]:
best_loss = np.inf
device = 'cuda'

In [31]:
for epoch in range(config.EPOCHS):
        train_loss = train_fn(train_dataloader, model, optimizer, scheduler)
        test_loss = eval_fn(val_dataloader, model)
        print(f"Train Loss = {train_loss} Valid Loss = {test_loss}")
        if test_loss < best_loss:
            torch.save(model.state_dict(), 'model.bin')
            best_loss = test_loss

  0%|          | 1/630 [00:00<06:20,  1.65it/s]

Train loss: 0.6421376466751099


  0%|          | 2/630 [00:01<05:54,  1.77it/s]

Train loss: 1.2166825532913208


  0%|          | 3/630 [00:01<05:40,  1.84it/s]

...

validation loss: 28.32806357741356
validation loss: 28.731754958629608


 97%|█████████▋| 68/70 [00:09<00:00,  7.07it/s]

validation loss: 29.17707660794258
validation loss: 29.587702989578247


100%|██████████| 70/70 [00:09<00:00,  7.03it/s]

validation loss: 30.053563207387924
validation loss: 30.365908563137054
Train Loss = 0.32422313311743356 Valid Loss = 0.4337986937591008


In [29]:
def train_fn(data_loader, model, optimizer, scheduler):
    model.train()
    total_train_loss = 0
    for i, batch in enumerate(tqdm(data_loader)):
        
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_labels = batch[2].cuda()
        outputs = model(b_input_ids,b_input_mask,labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_train_loss = total_train_loss + loss.item()
        # Zero the gradients
        optimizer.zero_grad()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        print('Train loss:' ,total_train_loss)
    return total_train_loss / len(data_loader)


In [30]:
def eval_fn(data_loader, model):
    model.eval()
    total_val_loss = 0
    for i, batch in enumerate(tqdm(data_loader)):
        
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_labels = batch[2].cuda()
        outputs = model(b_input_ids, 
                        b_input_mask,
                        labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_val_loss = total_val_loss + loss.item()
        print('validation loss:' ,total_val_loss)
        
    return total_val_loss / len(data_loader)

In [37]:
#

In [45]:
temp_model = XLMRobertaForTokenClassification.from_pretrained(
            config.BASE_MODEL)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [36]:
# temp_model.cuda()

In [49]:
outputs = temp_model(b_input_ids, 
                        attention_mask=b_input_mask,
                        labels=b_labels)

In [52]:
XLMRobertaForTokenClassification?

Init signature: XLMRobertaForTokenClassification(config)
Docstring:     
XLM-RoBERTa Model with a token classification head on top (a linear layer on top of the hidden-states output) e.g.
for Named-Entity-Recognition (NER) tasks.


This model inherits from :class:`~transformers.PreTrainedModel`. Check the superclass documentation for the generic
methods the library implements for all its model (such as downloading or saving, resizing the input embeddings,
pruning heads etc.)

This model is also a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`__
subclass. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to
general usage and behavior.

Parameters:
    config (:class:`~transformers.XLMRobertaConfig`): Model configuration class with all the parameters of the
        model. Initializing with a config file does not load the weights associated with the model, only the
        configuration. Check out the :met

In [53]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('xlm-roberta-base')
config.num_labels

2